---
title: IDs from Wind
order: 10
---

See following notebooks for details:

- [Magnetic field data](wind/mag.ipynb)
- [State data](wind/state.ipynb)

References:

- [HPDE.io/Wind](https://hpde.io/NASA/NumericalData/Wind/index.html)
- [Wind Data Sources](https://wind.nasa.gov/data_sources.php)
- [WIND MFI high-resolution data file](https://cdaweb.gsfc.nasa.gov/misc/NotesW.html#WI_H4-RTN_MFI)
- [PySPEDAS](https://pyspedas.readthedocs.io/en/latest/wind.html)

- We use magnetic field data in **RTN** coordinate system

Notes:

- December 2014: Wind Data Notice - Data between September 24 and November 30, 2014 were not included at this definitive data set because of instrumental problem [\*](https://omniweb.gsfc.nasa.gov/ftpbrowser/wind_swe_2m.html)


In [1]:
from space_analysis.utils.speasy import Variables
from discontinuitypy.datasets import IDsDataset
from discontinuitypy.utils.basic import resample
from beforerr.polars import pl_norm

from datetime import timedelta

from space_analysis.plasma.formulary import df_thermal_spd2temp

In [ ]:
from discontinuitypy.missions import wind_mag_h4_rtn_meta, wind_plasma_k0_swe_meta

In [7]:
timerange = ["2011-08-25", "2016-06-30"]
mission = "Wind"
ts = timedelta(seconds=1)
tau = timedelta(seconds=60)

provider = "archive/local"
mag_dataset = "WI_H4-RTN_MFI"
mag_parameters = ["BRTN"]
plasma_dataset = "WI_K0_SWE"
plasma_parameters = ["Np", "V_GSM", "THERMAL_SPD"]

## Downloading data file first


In [1]:
import pyspedas


def download(timerange):
    pyspedas.wind.threedp(timerange, datatype="3dp_pm", downloadonly=True)
    pyspedas.wind.mfi(timerange, datatype="h2", downloadonly=True)
    # pyspedas.wind.swe(timerange, datatype='k0', downloadonly=True)

## Loading


In [ ]:
def get_and_process_data(
    mag_dataset,
    mag_parameters,
    plasma_dataset,
    plasma_parameters,
    timerange,
    tau,
    ts,
    provider="archive/local",
):
    # define variables
    mag_vars = Variables(
        provider=provider,
        dataset=mag_dataset,
        parameters=mag_parameters,
        timerange=timerange,
    )

    plasma_vars = Variables(
        provider=provider,
        dataset=plasma_dataset,
        parameters=plasma_parameters,
        timerange=timerange,
    )

    # get column names
    bcols = mag_vars.data[0].columns
    density_col = plasma_vars.data[0].columns[0]
    vec_cols = plasma_vars.data[1].columns
    temperature_col = plasma_vars.data[2].columns[0]

    # get data
    mag_data = mag_vars.to_polars()
    plasma_data = (
        plasma_vars.to_polars()
        .with_columns(plasma_speed=pl_norm(vec_cols))
        .rename({density_col: "plasma_density"})
    )
    # process temperature data
    if plasma_vars.data[2].unit == "km/s":
        plasma_data = plasma_data.pipe(df_thermal_spd2temp, temperature_col)
    else:
        plasma_data = plasma_data.rename({temperature_col: "plasma_temperature"})

    return (
        IDsDataset(
            mag_data=mag_data.pipe(resample, every=ts),
            plasma_data=plasma_data,
            tau=tau,
            ts=ts,
            bcols=bcols,
            vec_cols=vec_cols,
            density_col="plasma_density",
            speed_col="plasma_speed",
            temperature_col="plasma_temperature",
        )
        .find_events(return_best_fit=False)
        .update_candidates_with_plasma_data()
    )